## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



In [1]:
import pandas as pd

In [2]:
def columns_csv(file, compression, sep):
    """
    To know which number fits with wich col name
    
    :param str file: Input file.
    :param str compression: Type of compression.
    :param str sep: Type o separator
    
    return: a dataframe with name of column and number
    """
    return pd.Series(pd.read_csv(file, compression=compression, header=None, nrows=1, sep=sep).T[0]).str.strip()

In [3]:
def cols_to_use(l_use, columns_csv):
    """
    List to columns to keep
    
    :param list l_use: List of str with the names of columns want to keep
    :param pd.Series columns_csv: Serie with the columns of the csv
    
    return: List with the number of column for selected column name
    """
    l = []
    for c in l_use:
        l.append(columns_csv[columns_csv == c].index[0])
    return l

In [4]:
booking_file = 'bookings.csv.bz2'
searches_file = 'searches.csv.bz2'
compression = 'bz2'

In [5]:
bookings_columns = columns_csv(booking_file, compression, '^')
searches_columns = columns_csv(searches_file, compression, '^')

In [6]:
booking_usecolumns = cols_to_use(['dep_port', 'arr_port', 'cre_date'], bookings_columns)
searches_usecolumns = cols_to_use(['Origin', 'Destination', 'Date'], searches_columns)

In [7]:
bookings_df = pd.read_csv(booking_file, compression=compression, sep='^', header=0, usecols=booking_usecolumns)

In [8]:
searches_df = pd.read_csv(searches_file, compression=compression, sep='^', header=0, usecols=searches_usecolumns)

In [10]:
bookings_columns

0      act_date
1        source
2      pos_ctry
3      pos_iata
4       pos_oid
5          rloc
6      cre_date
7      duration
8      distance
9      dep_port
10     dep_city
11     dep_ctry
12     arr_port
13     arr_city
14     arr_ctry
15     lst_port
16     lst_city
17     lst_ctry
18     brd_port
19     brd_city
20     brd_ctry
21     off_port
22     off_city
23     off_ctry
24     mkt_port
25     mkt_city
26     mkt_ctry
27         intl
28        route
29      carrier
30    bkg_class
31    cab_class
32     brd_time
33     off_time
34          pax
35         year
36        month
37          oid
Name: 0, dtype: object

In [11]:
searches_df.sample(2)

,Date,Origin,Destination
14266591,2013-09-23,MAD,SCL
14974862,2013-09-13,KUL,IST


In [12]:
bookings_df.sample(2)

,cre_date,dep_port,arr_port
6205927,2012-12-31 00:00:00,TLV,SFO
8553903,2013-09-23 00:00:00,ORD,CHA


In [13]:
bookings_df.describe(include='all')

,cre_date,dep_port,arr_port
count,10000010,10000010,10000010
unique,719,2148,2275
top,2013-01-14 00:00:00,LHR,LHR
freq,132250,268410,215551


In [14]:
searches_df.describe(include='all')

,Date,Origin,Destination
count,20390198,20390165,20390165
unique,360,2503,3068
top,2013-01-01,FRA,BKK
freq,57297,550736,477093


In [15]:
bookings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000010 entries, 0 to 10000009
Data columns (total 3 columns):
cre_date               object
dep_port               object
arr_port               object
dtypes: object(3)
memory usage: 228.9+ MB


In [16]:
searches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20390198 entries, 0 to 20390197
Data columns (total 3 columns):
Date           object
Origin         object
Destination    object
dtypes: object(3)
memory usage: 466.7+ MB


In [17]:
bookings_df.isnull().sum()

cre_date               0
dep_port               0
arr_port               0
dtype: int64

In [18]:
searches_df.isnull().sum()

Date            0
Origin         33
Destination    33
dtype: int64

## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

http://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [19]:
bookings_df['arr_port'].sample(2).str.len()

3376969    8
1091120    8
Name: arr_port, dtype: int64

In [21]:
bookings_df['dep_port'].head(10).str.len()

0    8
1    8
2    8
3    8
4    8
5    8
6    8
7    8
8    8
9    8
Name: dep_port, dtype: int64

In [22]:
# As we could see previously there are extra white space,
# so lets to clean that

In [23]:
bookings_df['arr_port'] = bookings_df['arr_port'].str.strip()
bookings_df['dep_port'] = bookings_df['dep_port'].str.strip()

In [24]:
bookings_df.sample(3)

,cre_date,dep_port,arr_port
2307291,2013-01-02 00:00:00,BNA,MIA
1600743,2013-01-16 00:00:00,DUS,MUC
2895870,2013-06-06 00:00:00,HAJ,PEK


In [25]:
# Anotther way to do the same but for all column is
# bookings_df = bookings_df.apply(lambda col: col.str.strip())

### * Duplicates

In [26]:
bookings_df.duplicated().sum()

9665133

In [27]:
searches_df.duplicated().sum()

20046020

In [28]:
bookings_df[bookings_df.duplicated()].head()

,cre_date,dep_port,arr_port
2,2013-03-26 00:00:00,SAL,CLT
4,2013-03-26 00:00:00,AKL,SVO
6,2013-03-20 00:00:00,DEN,LGA
8,2013-03-25 00:00:00,NRT,SIN
9,2013-03-25 00:00:00,NRT,SIN


In [29]:
%%time
bookings_df.drop_duplicates(inplace=True)

CPU times: user 5.77 s, sys: 521 ms, total: 6.29 s
Wall time: 6.59 s


###  Search

In [30]:
pd.options.display.max_columns = None
searches = pd.read_csv('searches.csv.bz2', sep='^', nrows=100000)
searches.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [31]:
bookings_df.tail()

,cre_date,dep_port,arr_port
999989,2013-05-14 00:00:00,POM,MEL
999991,2013-05-28 00:00:00,MEM,LAS
5000003,2013-03-26,AKL,SVO
5000007,60,JP,SG
5000008,"2013-03-25,00:00:00",TYO,SIN


In [32]:
len('2013-05-28 00:00:00')

19

In [37]:
bookings_df.columns = bookings_df.columns.str.strip()

In [38]:
bookings_df['clean_date'] = bookings_df['cre_date'].str[:10]
bookings_df = bookings_df[bookings_df['clean_date'].str.len() == 10]
bookings_df.shape

(334876, 4)

In [40]:
pd.to_datetime(bookings_df['cre_date'], errors='coerce').tail()

999985    2013-04-27
999989    2013-05-14
999991    2013-05-28
5000003   2013-03-26
5000008          NaT
Name: cre_date, dtype: datetime64[ns]

In [41]:
pd.to_datetime(bookings_df['cre_date'], errors='coerce').dropna().tail()

999984    2013-05-07
999985    2013-04-27
999989    2013-05-14
999991    2013-05-28
5000003   2013-03-26
Name: cre_date, dtype: datetime64[ns]

## 3) Make action plan

Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) remove duplicates

2) parse dates from string to datetime

3) remove whitespaces

        a) from colum names
        
        b) from content
        
4) remove NaN

5) define the model

    if there is one booking for a given O&D done at the same day as the search (for a given O&D), ALL searches of the day (for a given 0&D) might have resulted from the same source and will be set with 1.
    This is regardless of the boarding time of the plane... So if I was looking for plane for the first 4 days of December for a given O&D all those searches would be set to 1 not just the one correspoding to the correct boarding time

        match
        Search : [search_date, O&D] 
        Booking: [creation_date, O&D]

6) execute the model

        a) Group by bookings on [cre_date, O&D] so that we dont have duplicates 
        (and we can have number of bookings for the day)
        or we can just drop the duplicates
        b) search left join bookings adding "Booked" column
        c) test if the merge was done right
        d) fill NaN of "booked" column with 0
        e) pull all values of booked column >1 to 1

more complex... get number of segments from searches
match search_date, and then split all O&D of all segments, and match the date of first flight of each segment (seg1Date, seg2Date)
with boarding time and O&D and act_date of booking

What do we have?

In [42]:
bookings_df['booked'] = 1

joined = searches.merge(bookings_df, 
                        left_on=['Date', 'Origin', 'Destination'], 
                        right_on=['clean_date', 'dep_port', 'arr_port'], 
                        how='left')

joined['booked'].value_counts()

1.0    6404
Name: booked, dtype: int64

In [43]:
joined.drop(['cre_date', 'clean_date', 'dep_port', 'arr_port'], 
            axis=1, inplace=True)
joined['booked'] = joined['booked'].fillna(0).astype(int)

In [44]:
joined.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0


Why is it that the join has returned empty????


The data in two of the columns also has embedded whitespace, just like the column names.

In [45]:
%%time
import pandas as pd
pd.options.display.max_columns = None

bookings = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['cre_date           ', 'dep_port', 'arr_port'])
bookings.columns = bookings.columns.str.strip()
bookings = bookings.apply(lambda col: col.str.strip())
bookings['cre_date'] = bookings['cre_date'].str[:10]
bookings = bookings[bookings['cre_date'].str.len() == 10]
bookings.drop_duplicates(inplace=True)
bookings['booked'] = 1

searches = pd.read_csv('searches.csv.bz2', sep='^', nrows=100000)

joined = searches.merge(bookings, 
                        left_on=['Date', 'Origin', 'Destination'], 
                        right_on=['cre_date', 'dep_port', 'arr_port'], 
                        how='left')

joined.drop(['cre_date', 'dep_port', 'arr_port'], axis=1, inplace=True)
joined['booked'] = joined['booked'].fillna(0).astype(int)

CPU times: user 6min 45s, sys: 8.95 s, total: 6min 54s
Wall time: 6min 57s


Now it seems we got everything we need and not more: we haven't caused duplication or loss of records. We are ready to extend the approach to the whole dataset.

The problem text specifies that we need to write a csv out. Therefore, we don't even need to build the whole DataFrame in memory, we just need to write the partial results out as we get them. We can use `mode='a'` in order to append each chunk to the same file- we just need to be careful to write the header only once.

In [46]:
%%time
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = None


print('Starting with bookings at %s' % (datetime.now()))

bookings = pd.read_csv('bookings.csv.bz2', sep='^', usecols=['cre_date           ', 'dep_port', 'arr_port'])
bookings.columns = bookings.columns.str.strip()
bookings = bookings.apply(lambda col: col.str.strip())
bookings['cre_date'] = bookings['cre_date'].str[:10]
bookings = bookings[bookings['cre_date'].str.len() == 10]
bookings.drop_duplicates(inplace=True)
bookings['booked'] = 1

print('Finished with bookings at %s' % (datetime.now()))

searches = pd.read_csv('searches.csv.bz2', sep='^', chunksize=1000000)

for nchunk, chunk in enumerate(searches):
    print('Starting with chunk %.2d at %s' % (nchunk, datetime.now()))

    joined = chunk.merge(bookings, 
                            left_on=['Date', 'Origin', 'Destination'], 
                            right_on=['cre_date', 'dep_port', 'arr_port'], 
                            how='left')

    joined.drop(['cre_date', 'dep_port', 'arr_port'], axis=1, inplace=True)
    joined['booked'] = joined['booked'].fillna(0).astype(int)
    joined.to_csv('annotated_searches.csv.bz2', compression='bz2', mode='a')

Starting with bookings at 2018-12-20 10:01:46.456398
Finished with bookings at 2018-12-20 10:12:42.248881


<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


Starting with chunk 00 at 2018-12-20 10:13:06.102981


<string>:2: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.


Starting with chunk 01 at 2018-12-20 10:15:02.038485
Starting with chunk 02 at 2018-12-20 10:16:45.288111
Starting with chunk 03 at 2018-12-20 10:18:39.976326
Starting with chunk 04 at 2018-12-20 10:20:25.200461
Starting with chunk 05 at 2018-12-20 10:22:40.130573
Starting with chunk 06 at 2018-12-20 10:24:32.173042
Starting with chunk 07 at 2018-12-20 10:26:26.371971
Starting with chunk 08 at 2018-12-20 10:28:40.785162
Starting with chunk 09 at 2018-12-20 10:30:30.946627
Starting with chunk 10 at 2018-12-20 10:32:45.471736
Starting with chunk 11 at 2018-12-20 10:34:29.639015
Starting with chunk 12 at 2018-12-20 10:36:32.031886


<string>:2: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,44) have mixed types. Specify dtype option on import or set low_memory=False.


Starting with chunk 13 at 2018-12-20 10:38:19.681505
Starting with chunk 14 at 2018-12-20 10:40:15.606428
Starting with chunk 15 at 2018-12-20 10:41:59.460371
Starting with chunk 16 at 2018-12-20 10:43:44.113268
Starting with chunk 17 at 2018-12-20 10:45:33.852787
Starting with chunk 18 at 2018-12-20 10:47:34.431114
Starting with chunk 19 at 2018-12-20 10:49:31.681709
Starting with chunk 20 at 2018-12-20 10:51:13.011352
CPU times: user 47min 43s, sys: 1min 28s, total: 49min 11s
Wall time: 50min 4s
